<a href="https://colab.research.google.com/github/Lelouch-Vi-Britannia-Code-Geass/awesome-mysql/blob/master/DIP3180.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyqt5
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 kB 8.6 MB/s eta 0:00:00


In [ ]:
import os, sys
rpath = os.path.abspath(
    os.path.join(os.path.dirname(__file__), "..", "..")
)
sys.path.append(rpath)

import a_star
import dwa_paper_with_width as dwa

import math
import numpy as np
import matplotlib.pyplot as plt

show_animation = True
save_animation_to_figs = False

"""
In this version of codes, a lower resolution A* path is used to guide the DWA path.
Obstacles are then put onto the A* path, to test the performance of DWA.
The robot will head to the next local goal once it is close enough the current local goal.
v4 introduces a catch_localgoal_dist parameter that is larger than the catch_goal_dist.

In this version, the robot is a rectangle with width and length when specified,
    rather than always a circle, for collision check.
Obstacles are circles with radius.
"""

class Config:
    """
    simulation parameter class
    """

    def __init__(self):
        # robot parameter
        self.max_speed = 5.0  # [m/s]
        self.min_speed = 0.0  # [m/s]
        self.max_yaw_rate = 40.0 * math.pi / 180.0  # [rad/s]
        self.max_accel = 0.2  # [m/ss]
        self.max_delta_yaw_rate = 40.0 * math.pi / 180.0  # [rad/ss]
        self.v_resolution = 0.01  # [m/s]
        self.yaw_rate_resolution = 0.1 * math.pi / 180.0  # [rad/s]
        self.dt = 0.1  # [s] Time tick for motion prediction
        self.predict_time = 1.0  # [s]
        self.check_time = 100.0 # [s] Time to check for collision - a large number
        self.to_goal_cost_gain = 0.2
        self.speed_cost_gain = 1
        self.obstacle_cost_gain = 0.05
        self.robot_stuck_flag_cons = 0.001  # constant to prevent robot stucked
        self.robot_type = dwa.RobotType.rectangle
        self.catch_goal_dist = 0.5  # [m] goal radius
        self.catch_localgoal_dist = 1.0  # [m] local goal radius
        self.obstacle_radius = 0.5  # [m] for collision check

        # if robot_type == RobotType.circle
        # Also used to check if goal is reached in both types
        self.robot_radius = 0.5  # [m] for collision check

        # if robot_type == RobotType.rectangle
        self.robot_width = 0.5  # [m] for collision check
        self.robot_length = 1.2  # [m] for collision check

    @property
    def robot_type(self):
        return self._robot_type

    @robot_type.setter
    def robot_type(self, value):
        if not isinstance(value, dwa.RobotType):
            raise TypeError("robot_type must be an instance of RobotType")
        self._robot_type = value


config = Config()

# ----- Set up the map -----
ox, oy = [], []

# Border
for i in range(60):
    ox.append(i)
    oy.append(0.0)
for i in range(60):
    ox.append(60.0)
    oy.append(i)
for i in range(61):
    ox.append(i)
    oy.append(60.0)
for i in range(61):
    ox.append(0.0)
    oy.append(i)

# Inner Walls
# Vertical
for i in range(21):
    ox.append(10.0)
    oy.append(30.0+i)
for i in range(11):
    ox.append(20.0)
    oy.append(10.0+i)
for i in range(31):
    ox.append(30.0)
    oy.append(i)
for i in range(51):
    ox.append(40.0)
    oy.append(10.0+i)
# Horizontal
for i in range(10):
    ox.append(41.0+i)
    oy.append(10.0)
for i in range(10):
    ox.append(50.0+i)
    oy.append(20.0)
for i in range(20):
    ox.append(1.0+i)
    oy.append(20.0)
for i in range(20):
    ox.append(11.0+i)
    oy.append(30.0)
for i in range(10):
    ox.append(41.0+i)
    oy.append(30.0)
for i in range(20):
    ox.append(20.0+i)
    oy.append(40.0)
for i in range(10):
    ox.append(50.0+i)
    oy.append(40.0)
for i in range(20):
    ox.append(11.0+i)
    oy.append(50.0)


ob = np.array([ox, oy]).transpose()

# ----- Set up the start and goal positions -----
# Set the start and goal positions
sx, sy = 10.0, 10.0
gx, gy = 50.0, 50.0

# Plot the map
if show_animation:  # pragma: no cover
    if save_animation_to_figs:
        cur_dir = os.path.dirname(__file__)
        fig_dir = os.path.join(cur_dir, 'figs')
        os.makedirs(fig_dir, exist_ok=False)
        i_fig = 0
        fig_path = os.path.join(fig_dir, 'frame_{}.png'.format(i_fig))
    # plt.plot(ox, oy, ".k")
    for (x, y) in ob:
        circle = plt.Circle((x, y), config.robot_radius, color="k")
        plt.gca().add_patch(circle)
    plt.plot(sx, sy, "og")
    plt.plot(gx, gy, "*b")
    plt.grid(True)
    plt.axis("equal")

# ----- Run A* path planning -----
a_star_planner = a_star.AStarPlanner(
    ob, resolution=5.0, rr=1.0,
    min_x=min(*ox, sx-2, gx-2), min_y=min(*oy, sy-2, gy-2),
    max_x=max(*ox, sx+2, gx+2), max_y=max(*oy, sy+2, gy+2)
)
rx, ry = a_star_planner.planning(sx, sy, gx, gy)

road_map = np.array([rx, ry]).transpose()[::-1]
# print(road_map)

# Plot the path
if show_animation:  # pragma: no cover
    # plt.plot(rx, ry, "-r")
    plt.plot(rx, ry, "xb")
    plt.pause(0.001)

    if save_animation_to_figs:
        plt.savefig(fig_path)
        i_fig += 1
        fig_path = os.path.join(fig_dir, 'frame_{}.png'.format(i_fig))

# # ----- Put new obstacles on the A* path -----
# new_ob = np.array([
#     [12.5, 12.5],
#     [15.0, 17.5],
#     [15.0, 22.5],
#     [15.0, 27.5],
#     [15.0, 32.5],
#     [15.0, 37.5],
#     [17.5, 42.5],
#     [22.5, 42.5],
#     [27.5, 37.5],
#     [32.5, 32.5],
#     [35.0, 27.5],
#     [35.0, 22.5],
#     [37.5, 17.5],
#     [42.5, 17.5],
#     [45.0, 22.5],
#     [45.0, 27.5],
#     [45.0, 32.5],
#     [45.0, 37.5],
#     [45.0, 42.5],
#     [47.5, 47.5]
# ])
# new_ob1 = new_ob + np.array([0.5, 0.5])
# new_ob2 = new_ob + np.array([-0.5, -0.5])
# new_ob3 = new_ob + np.array([0.5, -0.5])
# new_ob4 = new_ob + np.array([-0.5, 0.5])
# new_ob = np.concatenate((new_ob1, new_ob2, new_ob3, new_ob4), axis=0)
# ob = np.append(ob, new_ob, axis=0)
# if show_animation:  # pragma: no cover
#     # plt.plot(new_ob[:,0], new_ob[:,1], ".k")
#     for (x, y) in new_ob:
#         circle = plt.Circle((x, y), config.robot_radius, color="k")
#         plt.gca().add_patch(circle)


# ----- Run DWA path planning -----
x = np.array([sx, sy, math.pi / 8.0, 1.0, 0.0])
# config = Config()

print(__file__ + " start!!")
trajectory = np.array(x)

if show_animation:  # pragma: no cover
    # for stopping simulation with the esc key.
    plt.gcf().canvas.mpl_connect(
        'key_release_event',
        lambda event: [exit(0) if event.key == 'escape' else None])
    plt_elements = []

for i_goal, dwagoal in enumerate(road_map):
    if i_goal == 0:  # Skip the start point
        continue

    while True:
        u, predicted_trajectory = dwa.dwa_control(x, config, dwagoal, ob)
        x = dwa.motion(x, u, config.dt)  # simulate robot
        trajectory = np.vstack((trajectory, x))  # store state history

        if show_animation:  # pragma: no cover
            for ele in plt_elements:
                ele.remove()
            plt_elements = []
            plt_elements.append(plt.plot(predicted_trajectory[:, 0], predicted_trajectory[:, 1], "-g")[0])
            plt_elements.append(plt.plot(x[0], x[1], "xr")[0])
            plt_elements.extend(dwa.plot_robot(x[0], x[1], x[2], config))
            plt_elements.extend(dwa.plot_arrow(x[0], x[1], x[2]))
            plt_elements.append(plt.plot(trajectory[:, 0], trajectory[:, 1], "-r")[0])
            plt.pause(0.001)

            if save_animation_to_figs:
                plt.savefig(fig_path)
                i_fig += 1
                fig_path = os.path.join(fig_dir, 'frame_{}.png'.format(i_fig))

        # check reaching goal
        dist_to_goal = math.hypot(x[0] - dwagoal[0], x[1] - dwagoal[1])
        if i_goal == len(road_map) - 1:
            if dist_to_goal <= config.catch_goal_dist:
                print("Goal!!")
                break
        else:
            if dist_to_goal <= config.catch_localgoal_dist:
                print("Local goal!!")
                break

print("Done")
if show_animation:  # pragma: no cover
    plt.show()




In [ ]:
from PyQt5 import *
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QComboBox
from PyQt5.QtGui import QFont
import sys
import math

font_title = QFont('Athelas', 16)
font_title.setBold(True)

font_button = QFont('Athelas', 12)


class Joystick(QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)

        # Set joystick properties
        self.joystick_radius = 25
        self.joystick_position = [100, 100]  # Starting position of the joystick (center of the widget)
        self.circle_radius = 100  # Radius of the circular joystick area
        self.setFixedSize(self.circle_radius * 2, self.circle_radius * 2)  # Size of the joystick area (diameter)

    def paintEvent(self, event):
        painter = QtGui.QPainter(self)
        painter.setRenderHint(QtGui.QPainter.Antialiasing)

        # Draw circular background (joystick area)
        painter.setBrush(QtGui.QBrush(QtGui.QColor(220, 220, 220)))
        painter.drawEllipse(0, 0, self.circle_radius * 2, self.circle_radius * 2)

        # Draw the joystick (movable circle)
        painter.setBrush(QtGui.QBrush(QtGui.QColor(50, 50, 250)))
        painter.drawEllipse(self.joystick_position[0] - self.joystick_radius,
                            self.joystick_position[1] - self.joystick_radius,
                            self.joystick_radius * 2,
                            self.joystick_radius * 2)

    def mouseMoveEvent(self, event):
        # Get the current mouse position relative to the widget
        x = event.x()
        y = event.y()

        # Calculate the distance from the center of the circle (circle_radius, circle_radius)
        distance = math.sqrt((x - self.circle_radius) ** 2 + (y - self.circle_radius) ** 2)

        # If the distance is within the circle radius, update the joystick position
        if distance <= self.circle_radius - self.joystick_radius:
            self.joystick_position = [x, y]
        else:
            # Restrict movement to the edge of the circle
            angle = math.atan2(y - self.circle_radius, x - self.circle_radius)
            self.joystick_position = [
                self.circle_radius + (self.circle_radius - self.joystick_radius) * math.cos(angle),
                self.circle_radius + (self.circle_radius - self.joystick_radius) * math.sin(angle)
            ]

        self.update()  # Trigger paintEvent to redraw the joystick

    def mouseReleaseEvent(self, event):
        # Return the joystick to the center when the mouse is released
        self.joystick_position = [self.circle_radius, self.circle_radius]
        self.update()


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # Set up the main window properties
        self.setWindowTitle("Navigation System")
        self.setGeometry(400, 200, 1200, 700)

        # No central widget layout is used here. We use absolute positioning.
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)

    #Command Label
        self.label_command = QtWidgets.QLabel(self.central_widget)
        self.label_command.setText("Command:")
        self.label_command.setFont(font_title)
        self.label_command.adjustSize()
        self.label_command.move(700, 30)

    #Command buttons
        self.btn_start = QtWidgets.QPushButton("Start", self.central_widget)
        self.btn_start.setFont(font_button)
        self.btn_start.move(700, 70)
        #Function for start

        self.btn_pause = QtWidgets.QPushButton("Pause", self.central_widget)
        self.btn_pause.setFont(font_button)
        self.btn_pause.move(820, 70)
        #Function for pause

        self.btn_manual = QtWidgets.QPushButton("Manual", self.central_widget)
        self.btn_manual.setFont(font_button)
        self.btn_manual.move(700, 110)
        #Function for manual

        self.btn_automatic = QtWidgets.QPushButton("Automatic", self.central_widget)
        self.btn_automatic.setFont(font_button)
        self.btn_automatic.move(820, 110)
        #Function for automatic

    #Joystick
        self.joystick = Joystick(self.central_widget)
        self.joystick.move(700, 160)  # Move joystick to desired position

    #Choose map
        self.map_select = QComboBox(self)
        self.map_select.setGeometry(1000, 70, 150, 30)

        self.map_select.addItem("Map 1")
        self.map_select.addItem("Map 2")
        self.map_select.addItem("Map 3")
        self.map_select.addItem("Map 4")
        self.map_select.addItem("Map 5")
        self.map_select.addItem("Customize")

    #Ship size
        self.ship_select = QComboBox(self)
        self.ship_select.setGeometry(1000, 120, 150, 30)

        self.ship_select.addItem("Small Ship")
        self.ship_select.addItem("Medium Ship")
        self.ship_select.addItem("Large Ship")

    #Velocity control
        self.label_control = QtWidgets.QLabel(self.central_widget)
        self.label_control.setText("Control:")
        self.label_control.setFont(font_title)
        self.label_control.adjustSize()
        self.label_control.move(980, 200)

        self.btn_accelerate = QtWidgets.QPushButton("Accelerate", self.central_widget)
        self.btn_accelerate.setFont(font_button)
        self.btn_accelerate.move(980, 240)
        #Function for accelerate

        self.btn_decelerate = QtWidgets.QPushButton("Decelerate", self.central_widget)
        self.btn_decelerate.setFont(font_button)
        self.btn_decelerate.move(980, 280)
        #Function for decelerate

    #Map
        self.label_map = QtWidgets.QLabel(self.central_widget)
        self.label_map.setText("Map:")
        self.label_map.setFont(font_title)
        self.label_map.adjustSize()
        self.label_map.move(50, 30)

        self.map = QWidget(self.central_widget)
        self.map.setStyleSheet("background-color: white; border: 1px solid black;")
        self.map.setGeometry(50, 80, 600, 570)

    #Dashboard
        self.btn_dashboard = QtWidgets.QPushButton("Dashboard", self.central_widget)
        self.btn_dashboard.setFont(font_button)
        self.btn_dashboard.move(700, 400)
        #Function for dashboard

        self.btn_notification = QtWidgets.QPushButton("Notification", self.central_widget)
        self.btn_notification.setFont(font_button)
        self.btn_notification.move(800, 400)
        #Function for notification

        self.btn_sensor_data = QtWidgets.QPushButton("Sensor data", self.central_widget)
        self.btn_sensor_data.setFont(font_button)
        self.btn_sensor_data.move(930, 400)
        #Function for sensor data

        self.win_dashboard = QWidget(self.central_widget)
        self.win_dashboard.setStyleSheet("background-color: white; border: 1px solid black;")
        self.win_dashboard.setGeometry(700, 430, 450, 220)
        #Function for dashboard display


def window():
    app = QApplication(sys.argv)
    win = MainWindow()
    win.show()
    sys.exit(app.exec_())


window()